<a href="https://colab.research.google.com/github/lezya/DeepLearning/blob/master/Kaggle_Grasp_and_Lift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM, BatchNormalization, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.optimizers import Adam
#from sklearn.metrics import roc_curve, auc, roc_auc_score
import tensorflow as tf
import glob

In [ ]:
path = 'train'
X_Paths = glob.glob('train/subj1_series1_data.csv')   #'train/subj*_series*_events.csv'
Y_Paths = glob.glob('train/subj1_series1_events.csv')
time_steps = 1000
subsample = 100

In [ ]:
x_data = []
y_data = []
for x_path in X_Paths:
  with open(x_path) as file:
    for line in file:
      if line[0] == "i": continue
      line_array = []
      for word in range(len(line.split(','))-1):
        word+=1
        line_array= np.append(line_array,(int(line.split(',')[word])))
        line_array = np.asarray(line_array)
      x_data.append(line_array)
x_data = np.asarray(x_data)   
for y_path in Y_Paths:
  with open(y_path) as file:
    for line in file:
      if line[0] == "i": continue
      line_array = []
      for word in range(len(line.split(','))-1):
        word+=1
        line_array= np.append(line_array,(int(line.split(',')[word])))
        line_array = np.asarray(line_array)
      y_data.append(line_array)
y_data = np.asarray(y_data)   

In [ ]:
x_data.shape

(119496, 32)

In [ ]:
model = Sequential()
#model.add(CuDNNLSTM(128, input_shape = (time_steps//subsample, 32)))
model.add(Conv2D(filters = 64, kernel_size = (7,7), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "same", activation = "relu", input_shape = (time_steps//subsample, 32, 1)))
model.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size = (3,3)))
model.add(Flatten())
#model.add(Dropout(0.2))
model.add(Dense(32, activation = "relu"))
model.add(BatchNormalization())
# model.add(Dropout(0.2))
model.add(Dense(6, activation = "sigmoid"))


adam = Adam(lr = 0.001)

model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ["accuracy"])

model.summary()

def generator(batch_size):
    while 1:
        
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(x_data)-time_steps)
            x_time_data[i] = x_data[random_index:random_index+time_steps:subsample]
            yy.append(y_data[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

model.fit_generator(generator(32), steps_per_epoch = 5000, epochs = 3)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 10, 32, 64)        3200      
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 32, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 32, 64)        102464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 32, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 32, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 10, 32, 64)        256       
_________________________________________________________________
flatten (Flatten)            (None, 20480)             0

KeyboardInterrupt: 

In [ ]:
x_val_path = "train/subj1_series2_data.csv"
y_val_path = "train/subj1_series2_events.csv"
x_val_data = []
with open(x_val_path) as file:
    for line in file:
        if line[0] == "i": continue
        line_array = []
        for word in range(len(line.split(','))-1):
            word+=1
            line_array.append(int(line.split(',')[word]))
        line_array = np.asarray(line_array)
        x_val_data.append(line_array)
x_val_data = np.asarray(x_val_data)    
        
y_val_data = []
with open(y_val_path) as file:
    for line in file:
        if line[0] == "i": continue
        line_array = []
        for word in range(len(line.split(','))-1):
            word+=1
            line_array.append(int(line.split(',')[word]))
        line_array = np.asarray(line_array)
        y_val_data.append(line_array)
y_val_data = np.asarray(y_val_data)

def val_generator():
    while 1:
        batch_size = 1
        x_time_data = np.zeros((batch_size, time_steps//subsample, 32))
        yy = []
        for i in range(batch_size):
            random_index = np.random.randint(0, len(x_val_data)-time_steps)
            x_time_data[i] = x_val_data[random_index:random_index+time_steps:subsample]
            yy.append(y_val_data[random_index + time_steps])
        yy = np.asarray(yy)
        yield x_time_data.reshape((x_time_data.shape[0],x_time_data.shape[1], x_time_data.shape[2], 1)), yy

In [ ]:
gen_data = val_generator()
scores = []
num_test = 1000
for i in range(num_test):
    x_test, y_test = next(gen_data)
    while not 1 in y_test:
#         print(y_test)
        x_test, y_test = next(gen_data)

#     print(y_test)
    y_out = model.predict(x_test).reshape((6,1))
    print(y_out)
#    scores.append(roc_auc_score(y_test.reshape((6,1)), y_out))
#scores = np.asarray(scores)
#print("Mean ", np.mean(scores))

[[7.9833684e-05]
 [6.6868059e-05]
 [7.4332952e-04]
 [2.1895766e-04]
 [9.2454839e-01]
 [9.9614346e-01]]
[[0.00465122]
 [0.6099438 ]
 [0.5983568 ]
 [0.6554974 ]
 [0.00239527]
 [0.00215623]]
[[0.09788516]
 [0.18995887]
 [0.19239241]
 [0.7519679 ]
 [0.00548664]
 [0.04516023]]
[[7.4304785e-06]
 [2.2576225e-05]
 [2.1025538e-04]
 [1.3521314e-04]
 [2.7805001e-02]
 [9.6854526e-01]]
[[1.2108645e-04]
 [1.6670227e-03]
 [7.8371465e-03]
 [2.3013741e-02]
 [2.4975300e-02]
 [4.4148853e-01]]
[[0.0004425 ]
 [0.01987073]
 [0.02260864]
 [0.01456901]
 [0.22430825]
 [0.14451548]]
[[0.00028586]
 [0.04237685]
 [0.04096609]
 [0.09116927]
 [0.00046638]
 [0.00444546]]
[[9.8108048e-06]
 [2.1049380e-04]
 [1.2810528e-03]
 [1.4821589e-03]
 [9.8799318e-01]
 [2.8794670e-01]]
[[5.4425338e-05]
 [1.8062294e-02]
 [1.4291313e-01]
 [6.3886511e-01]
 [1.8561482e-03]
 [1.7060935e-03]]
[[3.7096666e-05]
 [1.4236569e-04]
 [2.5573373e-04]
 [1.5705824e-04]
 [8.6116695e-01]
 [4.1396111e-01]]
[[8.0836962e-06]
 [1.6364455e-03]
 [7.0820

[[1.5196923e-05]
 [2.7957559e-04]
 [2.8334260e-03]
 [1.6897082e-02]
 [2.9717386e-03]
 [4.6514302e-01]]
[[2.6428699e-04]
 [3.7310338e-01]
 [3.1476077e-01]
 [6.7630434e-01]
 [5.0898105e-02]
 [7.2722435e-03]]
[[3.6183003e-05]
 [7.4014068e-04]
 [6.3584149e-03]
 [3.3687055e-02]
 [2.3695827e-03]
 [7.6479673e-02]]
[[5.7984194e-06]
 [2.5734240e-01]
 [3.8173997e-01]
 [8.0050415e-01]
 [5.9996186e-05]
 [6.2435865e-04]]
[[3.2030784e-05]
 [1.0235692e-05]
 [1.4802814e-04]
 [4.2849668e-05]
 [5.6883502e-01]
 [8.8447785e-01]]
[[9.8004763e-05]
 [5.7194203e-01]
 [7.1475399e-01]
 [3.9218864e-01]
 [2.4242699e-03]
 [1.3155043e-03]]
[[1.1550354e-05]
 [5.9299415e-05]
 [1.1806522e-04]
 [1.9559264e-04]
 [9.4278044e-01]
 [6.8461329e-01]]
[[9.4627263e-05]
 [1.3751715e-02]
 [3.1118691e-03]
 [9.0590119e-04]
 [1.8618762e-02]
 [2.2191107e-03]]
[[0.00316578]
 [0.3435353 ]
 [0.13131648]
 [0.17367086]
 [0.01730222]
 [0.04225382]]
[[1.5912514e-05]
 [8.3327293e-04]
 [4.5540929e-04]
 [1.5494227e-04]
 [2.1206945e-01]
 [2.75

[[3.3481508e-06]
 [4.0315294e-06]
 [2.6828551e-05]
 [4.4180233e-05]
 [4.1150320e-01]
 [9.1282058e-01]]
[[2.6295829e-06]
 [4.4298931e-06]
 [3.3188353e-05]
 [4.7670645e-05]
 [9.0554267e-01]
 [9.5670873e-01]]
[[0.01802704]
 [0.06190196]
 [0.00780445]
 [0.01691538]
 [0.00911176]
 [0.03936779]]
[[6.2863319e-06]
 [1.2081689e-01]
 [5.1915169e-02]
 [4.3195486e-03]
 [5.8641404e-02]
 [1.7633229e-02]]
[[7.0486451e-05]
 [1.1873931e-02]
 [2.5922775e-02]
 [8.9671940e-02]
 [2.8959066e-02]
 [2.8557122e-02]]
[[5.5721364e-05]
 [5.9342086e-03]
 [8.0842674e-03]
 [4.5166612e-03]
 [1.7466277e-02]
 [1.0380483e-01]]
[[2.5119802e-05]
 [3.7354231e-04]
 [9.6285343e-04]
 [1.1738539e-03]
 [9.6803904e-04]
 [4.2597562e-01]]
[[0.00233701]
 [0.02259693]
 [0.04923064]
 [0.01092428]
 [0.091782  ]
 [0.235194  ]]
[[0.00103366]
 [0.01224563]
 [0.02238753]
 [0.01398399]
 [0.11820176]
 [0.16248414]]
[[9.5171912e-05]
 [2.7071387e-02]
 [2.9880077e-02]
 [1.8681884e-03]
 [3.5964078e-01]
 [5.2988046e-01]]
[[0.01169661]
 [0.623947

[[0.00071105]
 [0.28018278]
 [0.13235709]
 [0.3846478 ]
 [0.00445771]
 [0.13691247]]
[[4.0455602e-06]
 [7.7155232e-04]
 [1.3569593e-03]
 [4.2349100e-04]
 [3.1924248e-04]
 [1.6499460e-02]]
[[7.0088786e-06]
 [3.7326515e-03]
 [7.1591437e-03]
 [5.6439131e-02]
 [5.2332878e-04]
 [2.2777736e-02]]
[[1.3807416e-04]
 [4.6268106e-04]
 [5.6982040e-04]
 [7.1805716e-04]
 [3.0990869e-01]
 [9.8287940e-02]]
[[6.6507389e-05]
 [5.7592988e-04]
 [3.8084388e-04]
 [4.7177076e-04]
 [2.4786443e-02]
 [1.7805845e-02]]
[[1.7482042e-04]
 [1.1861324e-01]
 [2.4295995e-01]
 [9.7897929e-01]
 [4.6968758e-03]
 [1.7735064e-03]]
[[6.8282147e-06]
 [4.3046843e-05]
 [4.2417645e-04]
 [2.6550889e-04]
 [9.9676275e-01]
 [9.3704307e-01]]
[[8.8004472e-06]
 [1.7126493e-05]
 [2.8511882e-04]
 [1.9874005e-06]
 [8.0711871e-02]
 [2.3721629e-01]]
[[0.00110412]
 [0.00111479]
 [0.00168023]
 [0.00362745]
 [0.7239282 ]
 [0.14802682]]
[[1.5948087e-06]
 [2.3705810e-02]
 [4.0244341e-02]
 [1.2441784e-02]
 [6.0800099e-01]
 [3.4954250e-03]]
[[1.07

[[1.3869112e-05]
 [1.7833412e-03]
 [1.5637279e-03]
 [6.9810748e-03]
 [1.3532814e-01]
 [7.5519681e-03]]
[[0.01930568]
 [0.44278547]
 [0.39425647]
 [0.13696367]
 [0.00075614]
 [0.00064078]]
[[0.02293977]
 [0.895746  ]
 [0.7527572 ]
 [0.26312563]
 [0.00428504]
 [0.00191617]]
[[1.1749680e-05]
 [4.4980556e-02]
 [1.9476950e-02]
 [1.4958590e-02]
 [3.2362342e-04]
 [6.2895715e-03]]
[[4.2206673e-05]
 [2.3826471e-01]
 [1.3285211e-01]
 [7.9646528e-02]
 [7.0700131e-05]
 [4.9220920e-03]]
[[9.4368617e-05]
 [3.0433190e-01]
 [2.9267094e-01]
 [6.4946622e-01]
 [3.2686889e-03]
 [4.3438077e-03]]
[[7.9600213e-05]
 [1.3531196e-10]
 [1.8480328e-05]
 [1.1561930e-10]
 [6.3134015e-05]
 [4.5021006e-01]]
[[0.00074798]
 [0.00933483]
 [0.0239481 ]
 [0.07547209]
 [0.00052682]
 [0.00376385]]
[[6.1114579e-02]
 [2.4688840e-03]
 [2.7632713e-04]
 [2.4065086e-05]
 [9.7676035e-05]
 [7.4836612e-04]]
[[5.1446896e-06]
 [2.5238454e-02]
 [2.1687120e-02]
 [1.8558800e-03]
 [3.1809449e-02]
 [2.9288232e-03]]
[[8.9259502e-06]
 [5.064

...

[[4.7531724e-04]
 [9.3441343e-01]
 [9.1859239e-01]
 [9.8970127e-01]
 [1.9007951e-02]
 [5.6677759e-03]]
[[0.2559555 ]
 [0.09289512]
 [0.11552021]
 [0.8441833 ]
 [0.01971948]
 [0.00734705]]
[[7.3823458e-07]
 [5.9860363e-06]
 [3.5722871e-05]
 [1.5017390e-04]
 [2.1692041e-01]
 [3.7047318e-01]]
[[8.9036330e-06]
 [3.7034417e-05]
 [7.6182703e-05]
 [9.2183363e-05]
 [4.9381623e-01]
 [2.4154937e-01]]
[[1.1573338e-06]
 [3.1880319e-02]
 [1.4223516e-01]
 [7.6368988e-01]
 [7.7658892e-04]
 [1.8379390e-03]]
[[0.53145534]
 [0.00984755]
 [0.01010633]
 [0.03264529]
 [0.00777569]
 [0.00256559]]
[[9.6926215e-06]
 [3.6540627e-04]
 [5.0252676e-04]
 [1.5217066e-03]
 [2.3868293e-02]
 [1.6852662e-01]]
[[0.07473925]
 [0.00706282]
 [0.01204959]
 [0.78854835]
 [0.02018437]
 [0.00165191]]
[[1.4665723e-04]
 [7.2601438e-04]
 [1.1526942e-03]
 [4.7913194e-04]
 [1.2303054e-01]
 [3.9850119e-01]]
[[1.9041609e-05]
 [9.2092156e-04]
 [4.5562685e-03]
 [1.5544981e-02]
 [7.1298484e-05]
 [9.0471417e-02]]
[[2.4000134e-05]
 [9.891

[[9.3280221e-05]
 [1.3807416e-04]
 [2.8508902e-04]
 [1.5100837e-04]
 [2.5298357e-01]
 [9.5939666e-02]]
[[0.00015825]
 [0.07685068]
 [0.03032622]
 [0.00744638]
 [0.00013775]
 [0.00014439]]
[[5.3086877e-04]
 [5.4374957e-01]
 [7.7164090e-01]
 [7.0078301e-01]
 [5.2011335e-01]
 [3.7915608e-01]]
[[9.7704960e-06]
 [1.4187370e-05]
 [7.2594215e-05]
 [5.0400391e-05]
 [9.7751164e-01]
 [9.0107530e-01]]
[[2.5768502e-05]
 [5.1933527e-04]
 [3.8635731e-04]
 [8.7568164e-04]
 [2.5586337e-02]
 [9.8964453e-02]]
[[0.01364228]
 [0.7746377 ]
 [0.9355071 ]
 [0.7585404 ]
 [0.9269979 ]
 [0.90782166]]
[[6.5947548e-05]
 [1.2948126e-02]
 [2.2522569e-02]
 [3.3715457e-02]
 [1.9443035e-04]
 [1.1263907e-02]]
[[3.9291382e-04]
 [7.0361751e-01]
 [8.7499559e-01]
 [5.5884832e-01]
 [2.1985507e-01]
 [1.1373699e-02]]
[[2.3687044e-05]
 [7.2428286e-03]
 [1.6917437e-02]
 [6.2144816e-02]
 [7.7330544e-05]
 [1.7438561e-02]]
[[3.8350714e-05]
 [5.1121801e-01]
 [7.8261238e-01]
 [8.8256758e-01]
 [6.2060356e-04]
 [1.5078187e-03]]
[[0.00

[[4.9593837e-05]
 [1.2676120e-03]
 [2.9588640e-03]
 [6.9832802e-04]
 [1.1077821e-03]
 [2.1685451e-02]]
[[0.00057402]
 [0.04615921]
 [0.03485584]
 [0.57343984]
 [0.02973834]
 [0.00802004]]
[[2.7164375e-05]
 [1.7836690e-04]
 [3.8543344e-04]
 [3.5354495e-04]
 [7.9465806e-03]
 [7.8642374e-01]]
[[7.4380827e-05]
 [4.7904003e-01]
 [2.0178735e-01]
 [7.7637881e-02]
 [2.1493435e-04]
 [3.9730046e-05]]
[[0.00177521]
 [0.00674021]
 [0.01496562]
 [0.00827637]
 [0.61480075]
 [0.9494447 ]]
[[1.0213494e-06]
 [2.1785498e-04]
 [1.0514557e-03]
 [1.5580922e-02]
 [6.0470402e-03]
 [4.5855135e-02]]
[[0.0037818 ]
 [0.07780868]
 [0.1281839 ]
 [0.29515815]
 [0.2841432 ]
 [0.07096016]]
[[6.3553068e-07]
 [4.6555765e-06]
 [2.2976472e-05]
 [3.4866778e-05]
 [1.2586209e-01]
 [5.2611196e-01]]
[[4.5911984e-06]
 [3.3408403e-04]
 [2.0085871e-03]
 [2.3837984e-03]
 [9.9051702e-01]
 [1.5024120e-01]]
[[0.00074959]
 [0.03176343]
 [0.09091282]
 [0.35975438]
 [0.00157973]
 [0.00076401]]
[[5.9659102e-05]
 [1.4002323e-02]
 [3.1072

[[7.2282790e-05]
 [1.0659308e-04]
 [7.3462725e-04]
 [5.4037571e-04]
 [9.9268818e-01]
 [8.7778747e-01]]
[[1.9197305e-05]
 [1.7389983e-02]
 [2.6975334e-02]
 [3.6408573e-02]
 [9.7078085e-04]
 [8.6441338e-03]]
[[2.90057033e-05]
 [1.05661034e-04]
 [1.83522701e-04]
 [5.82270441e-05]
 [1.84524775e-01]
 [6.89438701e-01]]
[[7.6726923e-05]
 [6.4833760e-03]
 [7.0461929e-03]
 [3.1870306e-03]
 [3.5966966e-01]
 [2.5047392e-01]]
[[8.4086128e-07]
 [1.6473919e-02]
 [5.2877754e-02]
 [4.8009977e-01]
 [1.5334559e-05]
 [8.8912249e-04]]
[[5.1594321e-05]
 [2.5048852e-04]
 [3.0165911e-04]
 [1.5968084e-04]
 [1.7176837e-02]
 [3.6007780e-01]]
[[5.9243084e-06]
 [1.2135804e-03]
 [3.3828318e-03]
 [1.9709024e-01]
 [6.3702309e-06]
 [1.8349290e-04]]
[[0.00013885]
 [0.0023998 ]
 [0.00501838]
 [0.03024906]
 [0.00679329]
 [0.03997046]]
[[2.8025623e-05]
 [1.6370416e-04]
 [2.8580427e-04]
 [1.7446280e-04]
 [1.4991403e-02]
 [7.8442901e-01]]
[[5.0998482e-05]
 [1.8578768e-04]
 [2.1344423e-04]
 [3.9997697e-04]
 [1.1049700e-01]


[[1.7112494e-03]
 [3.7217331e-16]
 [3.3861393e-11]
 [6.4478040e-01]
 [1.0000000e+00]
 [1.3903960e-10]]
[[8.0862374e-05]
 [8.0890357e-03]
 [2.5155336e-02]
 [9.5366538e-03]
 [4.8351198e-01]
 [7.2357327e-02]]
[[2.3766353e-07]
 [2.2005621e-05]
 [7.8147612e-05]
 [9.5865130e-04]
 [1.1551380e-03]
 [2.0733327e-02]]
[[0.00196618]
 [0.17280722]
 [0.07692781]
 [0.01838607]
 [0.00141808]
 [0.00076216]]
[[1.14116585e-04]
 [2.56675899e-01]
 [2.32064873e-01]
 [2.05742031e-01]
 [1.52796507e-04]
 [2.78818607e-03]]
[[0.0030238 ]
 [0.3876978 ]
 [0.57354134]
 [0.708437  ]
 [0.02830681]
 [0.01874688]]
[[1.0419557e-06]
 [1.5394085e-06]
 [5.1433308e-06]
 [2.3647162e-05]
 [2.9493988e-02]
 [3.3680010e-01]]
[[6.4150036e-06]
 [1.1097565e-05]
 [1.4952601e-05]
 [2.8308534e-05]
 [1.0234550e-01]
 [2.6306927e-01]]
[[0.0010772 ]
 [0.00336888]
 [0.03533223]
 [0.01190352]
 [0.00126588]
 [0.01560259]]
[[0.23867238]
 [0.0687533 ]
 [0.08951017]
 [0.09229591]
 [0.13448143]
 [0.03269583]]
[[0.00197703]
 [0.46541402]
 [0.6626